# Optimisation en architecture

***Kayané Robach***

***Magistère d'Economiste Statisticien***

***11 juin 2021***

Le but de ce projet est de construire et résoudre un problème d’optimisation appliqué à l’architecture. Initialement l'objectif était de proposer une alternative aux logements sociaux disposés en blocs. Pour cela nous définirons des contraintes à respecter, à savoir, minimiser le vis à vis, minimiser les surfaces de contact entre habitations, maximiser le champ visuel.

Nous nous intéressons donc au problème suivant :

Comment construire une structure agréable à vivre en utilisant un recuit simulé ?

Nous verrons dans un premier temps comment répondre à ce problème en 2D en utilisant la méthode du recuit simulé. Puis nous appliquerons ce processus en 3D pour répondre à notre problématique. Le choix de cette méthode mathématique s'est imposé au vue du cadre discret du problème et de la complexité de l'analyse. En effet je n'ai trouvé aucune autre méthode pertinente pour répondre aux objectifs choisis.

### Algorithme du recuit simulé

Soit $E$, l'espace d'état, fini et $H : E \longrightarrow \mathbb{R}$, la fonction objectif, une fonction dont on cherche un minimum.

On considère une mesure de Gibbs paramétrée par $T > 0$ de la forme $$\mu_T(x) = \frac{1}{Z_T} exp^{- \frac{H(x)}{T}}$$ où $Z_T$ est la constante de normalisation souvent appelée fonction de partition. Lorsque $T \longrightarrow 0$ la mesure $\mu_T$ se concentre sur les cubes de E proche des minimums de H ; on considère donc une suite $(T_n)_n$ telle que $\underset{n \longrightarrow \infty}{lim} T_n = 0$ et l'algorithme de Hastings-Metropolis afin trouver la solution minimum à notre problème.

___

**L'algorithme de Hastings-Metropolis** permet de construire une chaîne de Markov de probabilité invariante $\mu$ donnée ; en principe cet algorithme est utilisé pour simuler une loi (grâce à une chaine de Markov) qui est difficile à simuler.

L'algorithme construit de manière itérative une suite $(x_n)_n$ d'éléments de E. Le terme $x_n$ de la suite $(x_n)_n$ désigne le cube du chemin $(x_n)_n$ envisagé que nous sommes en train de construire et qui parcoure E dans un but précis (dépendant de chaque problème). Pour chaque nouvelle itération cherchons un cube suivant qui nous rapprochera de la solution du problème dont il est question. Nous sélectionnons donc un nouveau chemin candidat $(y_n)_n$ puis nous le testons afin de décider si nous le choisissons ou non comme étape suivante de notre chemin solution.

Une matrice $P$, dite de sélection, est définie afin de parcourir l'espace d'état E. Notre but est de construire une chaîne de Markov $(X_n)_n$ de probabilité invariante $\mu$. La matrice de sélection permet de sélectionner des candidats $y = (y_n)_n$ voisins de notre chemin actuel $x = (x_n)_n$ pour construire le chemin solution représenté par $(X_n)_n$. Ainsi $\forall x, y \in E, P(x,y) = \mathbb{P}(Y_n^{(x)} = y)$ défini la probabilité que l'on choisisse le chemin y en tant que candidat (la variable aléatoire du candidat étant $(Y_n)_n$) à partir du chemin actuel $x$.

Une fonction $\rho$ est ensuite définie à partir de $P$ et de $\mu$ par : $$\forall x,y \in E, \rho(x,y) = \frac{\mu(y) P(y,x)}{\mu(x) P(x,y)} \wedge 1$$ $\rho$ c'est la probabilité que l'on accepte le candidat $y$ selectionné par $P$.

A partir de toutes les définitions présentées nous pouvons ensuite construire la matrice stochastique Q comme la probabilité de procéder à la transition de x à y à l'étape n de la construction de notre suite solution. En termes mathématiques : $$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) \rho(x,y) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$

Sous les hypothèses adhoc, la chaîne de Markov $(X_n)_n$ (de transition Q) converge en loi vers $\mu$.

Pour résumer, on choisit un voisin $y$ de $x$ avec probabilité $P(x,y)$ (donnée par la matrice de sélection) puis, avec probabilité $\rho(x,y)$ on accepte ce voisin, auquel cas on pose $x_{n+1} = y$ ou on le refuse et on pose $x_{n+1} = x$

Concrètement la définition de notre chaîne de Markov est la suivante : (nous avons besoin de $(U_n)_n \approx \mathcal{U}_{[0,1]}$ pour construire $(X_n)_n$ rigoureusement) $$X_{n+1} = Y_{n+1}^{X_n} \mathbb{1}_{\{U_{n+1} \leq \rho(X_n,Y_{n+1}^{X_n})\}} + X_n \mathbb{1}_{\{U_{n+1} > \rho(X_n,Y_{n+1}^{X_n})\}}$$
___

Dans le cadre du recuit simulé la probabilité de procéder à la transition de x à y à l'étape n de la construction de notre suite solution : $Q$ est la suivante : $$Q_n(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) exp^{- \frac{(H(y) - H(x))_{+}}{T_n}} & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q_n(x,z) & \mbox{sinon.} \end{array} \right.$$

Voici un résumé schématique du fonctionnement du recuit simulé :

In [ ]:
from math import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.csgraph import connected_components
import datetime
from tqdm.notebook import tqdm
from IPython.display import Image

<img src="algo_scheme.png">

In [ ]:
light_colors = np.array(['thistle','plum','orchid','palevioletred','pink',
                         'cyan','magenta','green','olive','blue','yellow','red','lavender','grey','orange','aquamarine',
                         'lightgreen','lime','dimgrey','fuchsia','slateblue','plum','pink','peru','indigo',
                         'rosybrown','lightcoral','indianred','darksalmon','sandybrown','peachpuff','linen',
                         'lightgray','tan','darkorange','gold','darkkhaki','red','cyan','magenta',
                         'lightgreen','mediumaquamarine','paleturquoise','powderblue','skyblue','lightslategray',
                         'lightsteelblue','lavender'])

rng = np.random.default_rng()
  
def create_colored_matrix(matrix, colors):
    colors = rng.choice(light_colors, size=len(np.unique(matrix)))
    d = {}
    for value in np.unique(matrix):
        idx = int(value) 
        d[value] = colors[idx]
    Mcolor = matrix.copy()
    Mcolor = Mcolor.astype('str')
    for row in range(Mcolor.shape[0]):
        for col in range(Mcolor.shape[1]):
            for depth in range(Mcolor.shape[2]):
                key = float(Mcolor[row][col][depth])
                if int(key) == 0:
                    Mcolor[row][col][depth] = 'None'
                else :
                    Mcolor[row][col][depth] = d[key]
    return Mcolor, d

def colorate_new_matrix(matrix, d):
    Mcolor = matrix.copy()
    Mcolor = Mcolor.astype('str')
    for row in range(Mcolor.shape[0]):
        for col in range(Mcolor.shape[1]):
            for depth in range(Mcolor.shape[2]):
                key = float(Mcolor[row][col][depth])
                if int(key) == 0:
                    Mcolor[row][col][depth] = 'None'
                else :
                    Mcolor[row][col][depth] = d[key]
    return Mcolor

### Problème du petit lotissement

Sur un terrain carré nous voulons construire des maisons vitrées. Une contrainte doit être vérifiée : il doit y avoir le moins de vis à vis possible. Pour cela la première étape est d'éloigner les maisons au maximum les unes des autres.

Pour répondre à ce problème d'optimisation nous utiliserons l'algorithme du recuit simulé expliqué ci dessous.

Nous travaillons pour le moment en dimension 2.

### Modélisation de notre problème du petit lotissement

L'espace d'état E ici est une grille que l'on représentera par une matrice $k$ x $k$. Nous considèrerons $p$ cubes de cette grille défini chacun par 1 entier difféfent de 0 comme étant les maisons. La matrice "terrain" contient aisni des 0 pour les espaces libres et des entiers allant de 1 à $p$ pour les habitations.

Soit p maisons représentées par un vecteur $M = (M_1, M_2, M_3, ..., M_p)$ ; chaque entier représentant une maison se voit attribuer des coordonnées dans la matrice en 2 dimensions $\in [0,1,2, ..., k] \times [0,1,2, ..., k]$. Nous notons $d$ la distance euclidienne entre 2 maisons $M_i$ et $M_j$ $i,j \in {1,2,3, ..., p}$ : $d(M_i, M_j)$.

Notre algorithme devra retourner un grille remplie avec des maisons (donc une matrice de 0 avec un 1, un 2, un 3, ... et un $p$ pour représenter chaque maison dans la grille).

On souhaite maximiser la distance entre les maisons dans notre petit lotissement. On a donc un vecteur de distances $( d(M_1, M_2), d(M_1, M_3), d(M_2, M_3), ... )$ à maximiser.

H, la fonction objectif que nous allons minimiser, peut être donc être définie dans un premier temps comme l'opposé de la norme du vecteur des distances. Avec $p$ = 3 on a : $$H(M) = - ||(d(M_1, M_2), d(M_1, M_3), d(M_2, M_3))||$$

#### Code de la distance

On utilisera la distance euclidienne.

In [ ]:
def distance(matrix, i, j):
    """ 
    inputs : matrix (square matrix), i, j
    i and j are 2 integers different from 0. They identify cubes located in the matrix (0 = free space).
    The distance function returns the euclidean distance between the cubes i and j of the matrix. """
    cube_i_coordinate = np.asarray( matrix==i ).nonzero() # is a tuple, 1st element : array of the row coordinate, 2nd element : array of the column coordinate
    xi = cube_i_coordinate[0][0]
    yi = cube_i_coordinate[1][0]
    cube_j_coordinate = np.asarray( matrix==j ).nonzero() # is a tuple, 1st element : array of the row coordinate, 2nd element : array of the column coordinate
    xj = cube_j_coordinate[0][0]
    yj = cube_j_coordinate[1][0]
    return sqrt( (xi - xj)**2 + (yi - yj)**2 )

#### Code de H

Pour commencer, la fonction objectif est l'opposé de la norme du vecteur des distances. Si on veut maximiser les distances, on veut donc maximiser la norme du vecteur des distances. Cela revient à minimiser l'opposé.

In [ ]:
def H(matrix):
    """ 
    input : matrix (square matrix)
    matrix is fulfilled with 0 for free space and integers identifying each cubes.
    H function returns the opposit of the norm of a specific vector. It is the vector of the distances between 
    cubes in the matrix. Since it is our objective function, it should be interpreted as the distance between 
    occupied spaces in the matrix. """
    nbr_of_cubes = int(np.max(matrix))
    norm = np.array([])
    for i in range(1, nbr_of_cubes + 1):
        for j in range(i+1, nbr_of_cubes + 1):
            norm = np.append(norm, distance(matrix, i, j))
    return - np.linalg.norm(norm)

### Recuit simulé

Concrètement, nous avons une matrice avec des emplacements occupés et des emplacements libres. Nous choisissons une maison à déplacer. Nous cherchons les emplacements voisins de cette maison qui sont libres et nous sélectionnons un de ces espaces non occupés.

$ \left\{ \begin{array}{ll} \mbox{si } H \searrow & \mbox{nous déplaçons la maison }\\ \mbox{sinon } & \mbox{nous déplaçons la maison de temps en temps } \end{array} \right. $

Nous souhaitons explorer de temps en temps les alentours même si H ne diminue pas (d'où l'idée d'algorithme stochastique). Cela nous permettrait de sortir d'un minima local si nous nous retrouvions coincés.

Nous reviendrons plus en détails sur ce processus plus tard.

Définissons la matrice de sélection $P$ qui nous permettra de parcourir l'expace ainsi qu'une suite $(T_n)_n$ telle que $\underset{n \longrightarrow \infty}{lim} T_n = 0$.

Pour cela nous allons donner une matrice initiale à l’algorithme (représentant ici notre terrain avec quelques maisons). Nous choisissons une maison aléatoirement, puis avec une certaine probabilité nous déplaçons cette maison.



### Code de la suite T

Un théoreme énonce que si la matrice de sélection P est symétrique irréductible et si cette matrice vérifie la condition de Doeblin (faisant intervenir les chaînes de Markov et la théorie des probabilités) et si $$\forall n \geq 1, T_n = \frac{\gamma}{log(n)}$$ avec $\gamma > \underset{x, y}{max}(H(y) - H(x)) \mathbb{1}_{\{P(x,y)>0\}}$

Alors pour toute loi initiale $\nu_0$ de $X_0$ on a : $$\underset{n \longrightarrow \infty}{lim} ||\nu_n - \mu_n|| = 0$$ donc en particulier, $\underset{n \longrightarrow \infty}{lim} P( H(X_n)>argmin H ) = 0$

In [ ]:
def T(n):
    """ 
    inputs : n
    n is used to iterate in the Metropolis algorithm.
    The T function is the T sequence of the Metropolis Hastings theory. It is arbitrarely fixed here to 0.01/log(n). """
    return 0.01 / log(n)

### Code de la matrice de sélection

Pour le moment, étant donné une combinaison de position de maisons (un trajet dans le cas du problème du voyageur), on tire une maison aléatoirement, et on bouge cette maison dans un des espaces libres autour d'elle avec une certaine probabilité. Autour d'elle signifie dans un premier temps : à une norme 1 vallant 1.

Pour construire $P$ comme on le souhaite il faut d'abord une fonction qui identifie les espaces libres autour d'une maison.

Pour cela, nous créons une matrice d'indices (ou plutot 2 pour les indices lignes et les indices colonnes) que nous centrons avec les coordonnées de notre point.

Prenons un exemple : nous regardons le point qui devrait se trouver dans le carré rouge (point d'indice (2,2) car on commence à 0).

Nous construisons une matrice d'indices lignes et une matrice d'indices colonnes.

<img src="matrix_1.png" width="250px">

Nous centrons les matrices en soustrayant l'indice ligne à la matrice d'indices lignes et l'indice colonne à la matrice d'indices colonnes.

<img src="matrix_2.png" width="250px">

Nous calculons ensuite la somme des valeurs absolues des coordonnées des 2 matrices d'indices. Dans la matrice obtenue, les éléments $\leq$ 1 sont les voisins de notre point.

<img src="matrix_3.png" width="250px">

<img src="matrix_4.png" width="122px">

Nous pouvons ensuite construire $P$. Après sélection d'une maison et identification de l'espace libre autour de cette dernière, nous choisissons aléatoirement un emplacement parmi les emplacements libres et nous déplaçons la maison.

In [ ]:
def free_space_around_2d(matrix, cube):
    """ 
    inputs : matrix (square matrix), cube
    matrix is the space, should be an array of integers (0 = free space).
    cube is an integer (different from 0 and unique) in the matrix.
    The free_space_around_2d function gets the coordinates of the cubes at norm 1 = 1 around the cube in parameter.
    It return a tuple of 2 arrays : 
        - the first one for x coordinates
        - the second one for y coordinates
        of the cubes located at distance 1 with norm 1 of the cube. """
    cube_coordinate = np.asarray( matrix==cube ).nonzero() # is a tuple, 1st element : array of the row coordinate, 2nd element : array of the  col coordinate
    # we create 2 matrix of the row indexes and the columns indexes of the matrix in input
    # we center these matrix of indexes (with respect to the coordinate of our cube)
    # then each cube of norm 1 = 1 is a neighbor of our cube
    matrix_row_idx = np.repeat(np.arange(matrix.shape[0]).reshape(-1,1),matrix.shape[1],1)
    matrix_row_idx = np.abs(matrix_row_idx - cube_coordinate[0][0])
    matrix_col_idx = np.repeat(np.arange(matrix.shape[0]).reshape(-1,1),matrix.shape[1],1).transpose()
    matrix_col_idx = np.abs(matrix_col_idx - cube_coordinate[1][0]
    matrix_norm = (matrix_row_idx + matrix_col_idx) # is the norm 1 of each cube in the centered matrix of indexes
    # we return only cubes which are at norm 1 < 1 in the centered matrix of indexes 
    # and which are not occupied (value = 0)
    return np.asarray( (matrix_norm<=1)&(matrix==0) ).nonzero()

def P_2d(matrix):
    """ 
    input : matrix (square matrix) is the space, should be an array of integers (0 = free space).
    Only consider 2D problems !
    The P_2d function returns 
        - the new matrix with one cube relocated,
        - the new cube coordinates,
        - the old cube coordinates. """
    free_space = (np.array([]),np.array([]))
    while len(free_space[0]) <= 0:
        location = np.random.randint(1, np.max(matrix)+1) # is the identity integer of the cube we will move
        cube_coordinate = np.asarray( matrix==location ).nonzero() # is a tuple, 1st element : array of the row coordinate, 2nd element : array of the column coordinate
        row_coord = cube_coordinate[0][0]
        col_coord = cube_coordinate[1][0] # are the current coordinates of the cube we will move
        free_space = free_space_around_2d(matrix, location) # is a tuple with 2 arrays of row and column coordinates of the cubes around the one we want to move
    new_location_idx = np.random.randint(0, len(free_space[0])) # is the random place we choose in the free_space arrays to be the new location of our cube
    row_coord_new = free_space[0][new_location_idx]
    col_coord_new = free_space[1][new_location_idx]  
    # current place of the cube become 0 
    # new location become the identity integer of our cube
    matrix[row_coord,col_coord] = 0
    matrix[row_coord_new,col_coord_new] = location
    new_cube_coords = np.array([row_coord_new, col_coord_new])
    old_cube_coord = np.array([row_coord, col_coord])
    return matrix, new_cube_coords, old_cube_coord

Revenons à notre problème global, nous cherchons à minimiser une fonction H que nous connaissons car nous l'avons défini mais dont nous ne connaissons ni la forme ni le fond. Pour éviter les minima locaux nous faisons donc fluctuer notre avancée de manière aléatoire (en déplacant aléatoirement un cube à une place libre choisie aussi aléatoirement). C'est ainsi que nous seront capable d'aboutir au minimum de H (nous l'espérons).

Rappelons la définition de la marice de transition de notre problème de recuit simulé.

$$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) \rho(x,y) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$$$\rho(x,y) = \frac{\mu(y) P(y,x)}{\mu(x) P(x,y)} \wedge 1$$$$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) (\frac{\mu(y) P(y,x)}{\mu(x) P(x,y)} \wedge 1) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$$$\mu_{T_n}(x) = \frac{1}{Z_{T_n}} exp^{- \frac{H(x)}{T_n}}$$$$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) (\frac{\frac{1}{Z_{T_n}} exp^{- \frac{H(y)}{T_n}} P(y,x)}{\frac{1}{Z_{T_n}} exp^{- \frac{H(x)}{T_n}} P(x,y)} \wedge 1) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$
$P$ est supposée symétrique.

$$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) ( \frac{ exp^{ - \frac{H(y)}{T_n}} }{ exp^{- \frac{H(x)}{T_n}} } \wedge 1) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$$$Q(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) (exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} \wedge 1) & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q(x,z) & \mbox{sinon.} \end{array} \right.$$$$ exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n} } < 1 \iff exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n} } < exp^{0} \iff \frac{H(x)}{T_n} - \frac{H(y)}{T_n} < 0 $$$$ \implies \left\{ \begin{array}{ll} exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} \wedge 1 = 1 & \mbox{si } \frac{H(x)}{T_n} - \frac{H(y)}{T_n} \geq 0 \\ exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} \wedge 1 = exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} < 1 & \mbox{sinon.} \end{array} \right. $$$$ \iff \left\{ \begin{array}{ll} exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} \wedge 1 = exp^{0} = 1 & \mbox{si } (H(y) - H(x))_{+} = 0 \geq 0 \\ exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} \wedge 1 = exp^{- \frac{H(y)}{T_n} + \frac{H(x)}{T_n}} < 1 & \mbox{sinon.} \end{array} \right. $$$$Q_n(x,y) = \mathbb{P}[X_{n+1} = y | X_n = x] = \left\{ \begin{array}{ll} P(x,y) exp^{- \frac{(H(y) - H(x))_{+}}{T_n}} & \mbox{si } x \neq y \\ 1 - \sum_{z \neq x} Q_n(x,z) & \mbox{sinon.} \end{array} \right.$$
Concrètement que doit-il se passer ?

On a notre matrice avec une certaine combinaison d'emplacements occupés et d'emplacements libres. On choisi aléatoirement une maison à déplacer (grâce à la matrice de sélection $P$) et, on choisi le nouvel emplacement qu'elle va éventuellement occuper (parmi les emplacements voisins de cette maison). La norme utilisée a ici un rôle très important dans la définition des espaces voisins disponibles et/ou occupés. Avec une certaine probabilité (définie par $\rho$) nous déplaçons cette maison. Comment ?

Nous avons là défini l'éventuel nouvel emplacement de notre maison. Nous comparons alors la valeur de notre fonction objectif $H$ évaluée à l'ancienne matrice puis à la nouvelle (après éventuel délacement d'une des maisons).

- Si H diminue avec le déplacement nous déplaçons bien la maison :

Si $H(new\_matrix) < H(matrix)$, alors nous déplaçons la maison.

- Si H ne diminue pas avec le déplacement : $H(new\_matrix) > H(matrix)$ $\iff$ $\frac{H(matrix) - H(new\_matrix)}{T(n)} < 0$ $\iff$ $exp^{\frac{H(matrix) - H(new\_matrix)}{T(n)}} < 1$, nous souhaitons quand même explorer de temps en temps les alentours donc nous voulons déplacer la maison les $exp^{\frac{H(matrix) - H(new\_matrix)}{T(n)}}$ du temps :

Si $\mathcal{U}_{[0,1]}$ < $exp^{ \frac{H(matrix) - H(new\_matrix)}{T(n)} }$, alors nous déplaçons la maison.

- Sinon nous ne déplaçons pas la maison.

### L'Algorithme de Metropolis

In [ ]:
def transition_Q(H, P, T, matrix, H_matrix, n):
    """
    inputs : 
    H the objective function to minimize, 
    P the functionjapplying the selection matrix to move one cube,
    T the sequence converging towards 0 (optimaly : epsilon / log(n)), 
    matrix (square matrix), 
    n the iteration step.
    The function is in charge of moving (or not) a cube selected with P as explained above. """
    new_matrix, new_cube, old_cube = P(np.copy(matrix))
    H_new_matrix = H(new_matrix)
    delta = H_new_matrix - H_matrix
    if delta < 0:
        return new_cube, new_matrix, H_new_matrix
    elif np.random.uniform(0,1) < exp(- delta / T(n)):
        return new_cube, new_matrix, H_new_matrix
    else:
        return old_cube, matrix, H_matrix
    
def Metropolis(H, P, T, matrix, N):
    """
    inputs : 
    H the objective function to minimize, 
    P the functionjapplying the selection matrix to move one cube,
    T the sequence converging towards 0 (optimaly : epsilon / log(n)), 
    matrix (square matrix), 
    N the total number of iterations.
    The function iterates N times the transition defined above. So, we "progress" thanks to the N steps towards the minimum. """
    objective = []
    new_matrix = np.copy(matrix)
    H_new_matrix = H(np.copy(matrix))
    for n in tqdm(range(2,N)):
        new_cube, new_matrix, H_new_matrix = transition_Q(H, P, T, new_matrix, H_new_matrix, n)
        objective.append(H_new_matrix)
    return objective, new_matrix

Let's go !

Nous lançons l'algorithme sur une matrice 10x10 en 2D.

In [ ]:
M = np.zeros((10,10,1)) # etage, ligne, colonne
M[2][4][0] = 1
M[3][4][0] = 2
M[1][4][0] = 3
M[5][4][0] = 4
M[4][4][0] = 5
Mcolor, dcolor = create_colored_matrix(M, light_colors)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
ax.voxels(M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

N0 = 1000
distances, new_M = Metropolis(H, P_2d, T, M, N0)
Mcolor = colorate_new_matrix(new_M, dcolor)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "grey", alpha = 0.7)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/100))])
plt.title("Evolution de H au cours des itérations")
plt.show()

<img src="simulation_1.png" width="500px">

<img src="simulation_2.png" width="500px">

<img src="H_evolution_1.png" width="500px">

Un problème apparaît. Notre fonction H étant définie comme elle l'est, c'est à dire, en prenant en compte la distance moyenne des maisons aux autres, l'algorithme n'a pas d'intérêt à changer une combinaison comme celle qui apparaît à la fin. En effet, si on déplace une des maisons qui est acollée à une autre, notre fonction objectif n'est pas modifiée.

Nous allons donc changer notre fonction objectif.

Petit retour en arrière pour redéfinir H.

### Code de H

La fonction objectif est maintenant la somme des inverses des distances de chaque maison aux autres. Si on veut maximiser les distances, on veut donc minimiser les inverses des distances. Cela permet de mettre plus de poids sur les distances courtes et ainsi de faire en sorte que chaque maison s'isole des autres.

In [ ]:
def H(matrix):
    """ 
    input : matrix (square matrix)
    matrix is fulfilled with 0 for free space and integers identifying each cubes.
    H function returns the sum of the inverse of the distances of each building with respect to another. """
    nbr_of_cubes = np.max(matrix)
    sum_of_inverse_distance = np.array([])
    for i in range(1, int(nbr_of_cubes) + 1):
        for j in range(i+1, int(nbr_of_cubes) + 1):
            sum_of_inverse_distance = np.append(sum_of_inverse_distance, 1 / distance(matrix, i, j))
    return sum(sum_of_inverse_distance)

Voyons si nous avons réussi améliorer notre algorithme :

In [ ]:
Mcolor, dcolor = create_colored_matrix(M, light_colors)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
ax.voxels(M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

N0 = 1000
distances, new_M = Metropolis(H, P_2d, T, M, N0)
Mcolor = colorate_new_matrix(new_M, dcolor)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/100))])
plt.title("Evolution de H au cours des itérations")
plt.show()

<img src="simulation_3.png" width="500px">

<img src="simulation_4.png" width="500px">

<img src="H_evolution_2.png" width="500px">

Regardons ce que ca donne quand la moitié de la surface est occupée :

In [ ]:
# etage, ligne, colonne
Mx = np.arange(10)
My = np.arange(10)
M = np.zeros((10,10, 1))
for i in range(1,M.shape[0]+1):
    places_y = np.random.choice(My, int(M.shape[0]/3), replace=False)
    for j in range(1,len(places_y)+1):
        M[i-1][places_y[j-1]][0] = j + int(M.shape[0]/3)*(i-1)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
Mcolor, dcolor = create_colored_matrix(M, light_colors)
ax.voxels(M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

N0 = 1000
distances, new_M = Metropolis(H, P_2d, T, M, N0)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.set_box_aspect((4,4,0.4))
Mcolor = colorate_new_matrix(new_M, dcolor)
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/100))])
plt.title("Evolution de H au cours des itérations")
plt.show()

<img src="simulation_5.png" width="500px">

<img src="simulation_6.png" width="500px">

<img src="H_evolution_3.png" width="500px">

### Problème du HAV, l'Habitation Agréable à Vivre

Nous avons vu le problème du petit lotissement ; fini les plaisanteries, nous passons désormais en 3D. Sur un terrain carré (ou plutôt cubique car nous pouvons considérer la hauteur), nous voulons construire un batiment.

Ce sera une structure où chacun des cubes qui la composent représente une habitation. Comme tout à l'heure nous imaginons des petites habitations vitrées. Mais attention, nous changeons les contraintes : il doit toujours y avoir le moins de vis à vis possible et nous voulons que les habitations ne forment pas un block, nous souhatons une structure plus épurée. Nous allons définir proprement toutes ces contraintes par la suite.

Nous travaillons désormais en 3D ; dans notre matrice initiale, tous les cubes sont au sols.

### Voici notre matrice initiale

In [ ]:
M = np.zeros((10,10,10)) # etage, ligne, colonne
M[3][4][0] = 1
M[4][4][0] = 2
M[5][4][0] = 3
M[6][4][0] = 4
M[3][5][0] = 5
M[4][5][0] = 6
M[5][5][0] = 7
M[6][5][0] = 8
M[3][4][1] = 9
M[4][4][1] = 10
M[5][4][1] = 11
M[6][4][1] = 12
M[3][5][1] = 13
M[4][5][1] = 14
M[5][5][1] = 15
M[6][5][1] = 16
Mcolor, dcolor = create_colored_matrix(M, light_colors)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
ax.voxels(M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

<img src="simulation_7.png" width="500px">

Pour répondre à nos besoins nous définissons quelques fonctions utiles.

- un processus pour identifier les places libres dans la matrice qui se trouvent autour d'un cube, ainsi que les places occupées qui se trouvent autour d'un cube.
Nous avons besoin de 4 fonctions pour cela ; get_matrix_indexes fait, tout comme en 2D, une matrice d'indices, que cette fois nous représentons en tuple (x, y, z). centered_matrix_norm existe en 2 exemplaire, un pour la norm 1 et un pour la norm 2. Tout comme en 2D encore, ces fonctions centrent la matrice des indices à partir des coordonnées d'un cube spécifique afin d'identifier les voisins du cube dont il est question. Enfin get_location_around sert à la fois à identifier les emplacements libres et les emplacements occupées autour du cube passé en paramètre.

- une fonction pour créer une matrice d'affinité entre les cubes présents dans notre grille 3D. Vous vous demanderez, mais pourquoi ? Nous créons une matrice d'affinité entre cubes pour identifier les "affinités" qu'ils ont entre eux, c'est à dire les liens. Cette fonction qui nous ressort donc une matrice remplie de 1 et de 0 indiquants un lien ou non entre 2 cubes en ligne et colonne. Nous l'utilisons par la suite pour identifier le nombre de groupes de cubes (à partir de la théorie des graphes). Ces groupes de cubes seront définis en focntions des affinités, elles mêmes définies en fonction de la norme.

- Une fonction qui calcule "le champ visuel". Elle retourn 2 éléments, le champ Nord-Sud et Est-Ouest. Le champ visuel est défini comme le compte des places libres autour d'un cube mais uniquement dans l'axe des abcisses ou des ordonnées. Vous comprendrez mieux avec un petit dessin :

<img src="simulation_8.png" width="500px">

In [ ]:
def get_matrix_indexes(matrix):
    """ 
    Inputs : matrix (square matrix)
    Returns a matrix filled with tuple of 3 elements (x,y,z) of the indexes of each possible cube location in 3d. """
    idx = np.zeros(matrix.shape,dtype='i,i,i')
    for x in range(matrix.shape[0]):
        for y in range(matrix.shape[1]):
            for z in range(matrix.shape[2]):
                idx[x][y][z] = (x,y,z)
    return idx

def centered_matrix_norm1(matrix, cube):
    """ 
    Inputs : matrix (square matrix), cube
    Returns a norm 1 matrix (built as we did in 2d). Using a matrix of tuples indexes in 3d 
    we center the matrix using the cube coordinates and we compute the norm 1.
    Then the norm 1 matrix is returned. """
    idx = get_matrix_indexes(matrix)
    cube_coordinate = np.asarray(matrix==cube).nonzero()
    cube_tuple = (cube_coordinate[0][0],cube_coordinate[1][0],cube_coordinate[2][0])
    norm = np.zeros(matrix.shape,dtype='i')
    for x in range(matrix.shape[0]):
        for y in range(matrix.shape[1]):
            for z in range(matrix.shape[2]):
                idx[x][y][z][0] = abs(idx[x][y][z][0] - cube_tuple[0])
                idx[x][y][z][1] = abs(idx[x][y][z][1] - cube_tuple[1])
                idx[x][y][z][2] = abs(idx[x][y][z][2] - cube_tuple[2])
                norm[x][y][z] = sum(idx[x][y][z])                
    return norm

def centered_matrix_norm2(matrix, cube):
    """ 
    Inputs : matrix (square matrix), cube
    Returns a norm 2 matrix. Using a matrix of tuples indexes in 3d 
    we center the matrix using the cube coordinates and we compute the norm 2.
    Then the norm 2 matrix is returned. """
    idx = get_matrix_indexes(matrix)
    cube_coordinate = np.asarray(matrix==cube).nonzero()
    cube_tuple = (cube_coordinate[0][0],cube_coordinate[1][0],cube_coordinate[2][0])
    norm = np.zeros(matrix.shape,dtype='i')
    for x in range(matrix.shape[0]):
        for y in range(matrix.shape[1]):
            for z in range(matrix.shape[2]):
                idx[x][y][z][0] = abs(idx[x][y][z][0] - cube_tuple[0])**2
                idx[x][y][z][1] = abs(idx[x][y][z][1] - cube_tuple[1])**2
                idx[x][y][z][2] = abs(idx[x][y][z][2] - cube_tuple[2])**2
                norm[x][y][z] = sqrt(sum(idx[x][y][z]))          
    return norm

def get_location_around(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use):
    """ 
    Inputs : matrix (square matrix), cube, norm_value_contact, norm_value_free
    Look for the location around the cube in the matrix which are close (according to the norm 1 as expressed in centered_matrix_norm) to the cube.
    Returns 2 matrixes. The first one, matrix filled with True/False for the occupied locations in contact with 
    the cube (at norm norm_value_contact from the cube). The second one, matrix filled with True/False for the free location around the cube
    (at norm norm_value_free from the cube). """
    if str_norm_to_use == "norm1":
        centered_matrix_norm = centered_matrix_norm1
    elif str_norm_to_use == "norm2":
        centered_matrix_norm = centered_matrix_norm2
    else :
        print("error norm, we will use norm1")
        centered_matrix_norm = centered_matrix_norm1
    norm = centered_matrix_norm(matrix, cube)
    # get occupied location in contact with the cube :
    contact_spaces = np.asarray( (norm<=norm_value_contact)&(matrix!=0)&(matrix!=cube) ).nonzero()
    contact = np.full(matrix.shape, False)
    for coords in range(len(contact_spaces[0])):
        contact[contact_spaces[0][coords]][contact_spaces[1][coords]][contact_spaces[2][coords]] = True
    # get location around the cube which are free :
    free_spaces = np.asarray( (norm<=norm_value_free)&(matrix==0) ).nonzero()
    free = np.full(matrix.shape, False)
    for coords in range(len(free_spaces[0])):
        free[free_spaces[0][coords]][free_spaces[1][coords]][free_spaces[2][coords]] = True    
    return contact, free

def create_affinity_matrix(matrix, norm_value_contact, norm_value_free, str_norm_to_use):
    """ 
    Inputs : matrix (square matrix), norm_value_contact, norm_value_free, str_norm_to_use
    (norm_value_contact, norm_value_free and str_norm_to_use are parameters for the get_location_around)
    Create an affinity matrix of the cubes in the matrix passed in parameter.
    Since then, it is a symetric matrix of shape (number of cubes, number of cubes). """
    affinity = np.zeros( (int(np.max(matrix)),int(np.max(matrix))) )
    for cube in range(1, int(np.max(matrix))+1):
        cube_coord = np.asarray( matrix==cube ).nonzero()
        in_contact, free = get_location_around(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use)
        for other_cube in matrix[in_contact]:
            affinity[cube-1, int(other_cube)-1] = 1
    return affinity

def get_visual_field_length(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use):
    """ 
    Inputs : matrix (square matrix), cube, norm_value_contact, norm_value_free, str_norm_to_use
    (norm_value_contact, norm_value_free and str_norm_to_use are parameters for the get_location_around)
    We look at a cube (passed in parameter) and we return what we call the visual field (South North) and (East West).
    We define the visual field (S/N or E/W) as the count of free spaces on the same level (coordinate z) of the cube
    in S/N directions before meeting another cube. """
    nbr_of_cubes = np.max(matrix)
    # we look at free spaces around
    # we look at spaces occupied around
    cubes_in_contact, cubes_free_locations = get_location_around(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use)
    cubes_in_contact_coord = np.asarray( cubes_in_contact==True ).nonzero()
    cubes_free_locations_coord = np.asarray( cubes_free_locations==True ).nonzero()
    # we place ourselve at the floor where the cube is located (z coordinate) :
    fix_z = np.asarray( matrix==cube ).nonzero()[2][0]
    # visual field = free spaces before the next cube
    fix_x = np.asarray( matrix==cube ).nonzero()[0][0]
    fix_y = np.asarray( matrix==cube ).nonzero()[1][0]
    # we look at the visual field Sud/Nord, for fixed z and fixed x :
    visual_field = np.full(matrix.shape, False)
    for y in range(matrix.shape[1]):
        if y != fix_y :
            if matrix[fix_x][y][fix_z] == 0:
                visual_field[fix_x][y][fix_z] = True
    visual_N = visual_field[fix_x,:,fix_z][np.asarray( matrix==cube ).nonzero()[1][0]+1:]
    visual_S = visual_field[fix_x,:,fix_z][:np.asarray( matrix==cube ).nonzero()[1][0]-1]
    visual_S_N = 0
    for element in visual_N:
        if element == True:
            visual_S_N += 1
        else:
            break
    for element in visual_S:
        if element == True:
            visual_S_N += 1
        else:
            break
    # we look at the visual field Est/West, for fixed z and fixed y :
    visual_field = np.full(matrix.shape, False)
    for x in range(matrix.shape[0]):
        if x != fix_x :
            if matrix[x][fix_y][fix_z] == 0:
                visual_field[x][fix_y][fix_z] = True
    visual_W = visual_field[:,fix_y,fix_z][np.asarray( matrix==cube ).nonzero()[0][0]+1:]
    visual_E = visual_field[:,fix_y,fix_z][:np.asarray( matrix==cube ).nonzero()[0][0]-1]
    visual_W_E = 0
    for element in visual_W:
        if element == True:
            visual_W_E += 1
        else:
            break
    for element in visual_E:
        if element == True:
            visual_W_E += 1
        else:
            break
    return visual_S_N, visual_W_E

Grâce aux fonctions ci dessus nous allons définir nos objectifs. Nous voulons désormais minimiser les surfaces en contact et maximiser le champ visuel.

Premièrement nous voulons minimiser les surfaces en contact entre cubes, donc H prend en compte la moyenne des surface en contact par cube. Ensuite nous voulons maximiser le champ visuel, donc minimiser l'inverse du champ visuel. Rappelez vous comment nous avons défini le champ visuel : il est le total des cases séparant la face d'un cube et le prochain cube rencontré sur le même axe (pour une même hauteur). Pour être plus explicit, imagnez un cube-appartement traversant (avec des fenêtres sur 2 faces opposées), vous vous tenez au milieu de la pièce et regardez une fenêtre puis vous comptez l'espace entre votre appartement-cube et le suivant qui se trouve dans votre champ de vision ; en sommant ces valeurs pour chacune des fenêtres nous obtenons ce que nous avons appelé ici le champ visuel.

Mais il y a 2 champs visuels, le Nord-Sud et le Est-Ouest. A chaque fois, nous avons donc décidé de considérer comme le champ visuel à maximiser, celui qui est le plus grand.

Ainsi H est la somme de :

- la moyenne des surfaces en contact des cubes
- l'inverse de la moyenne des champs visuels des cubes

### Code de H

In [ ]:
def H(matrix):
    """ 
    input : matrix (square matrix)
    (norm_value_contact, norm_value_free, str_norm_to_use are needed for the get_location_around function)
    matrix is fulfilled with 0 for free space and integers identifying each cubes.
    H function returns the mean of the number of cube in contact with each cube + the inverse of the mean of visual field of each cube. """
    nbr_of_cubes = np.max(matrix)
    nbr_cubes_in_contact = np.array([])
    visual_field = np.array([])
    for i in range(1, int(nbr_of_cubes) + 1):
        start=datetime.datetime.now()
        cube = i
        cubes_in_contact, cubes_free_locations = get_location_around(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use)
        nbr_cubes_in_contact = np.append(nbr_cubes_in_contact, sum(sum(sum(cubes_in_contact))))
        field_SN, field_WE = get_visual_field_length(matrix, cube, norm_value_contact, norm_value_free, str_norm_to_use)
        visual_field = np.append(visual_field, np.maximum(field_SN, field_WE))
    return np.mean(nbr_cubes_in_contact) + 1/np.mean(visual_field)

C'est parti pour définir la matrice de passage P cette fois en 3D.

Comment faire ?

Nous piochons un cube et nous identifions les espaces libres et occupés qu'il a autour de lui. (Et s'il n'y en a pas nous en piochons un autre).

Là, nous allons proposer une nouvelle place pour le cube sélectionné, et nous allons calculer le nombre de groupes de cubes qui seraient alors identifiés (grâce à la matrice d'affinité), si nous déplacions le cube. Rappelez vous, nous souhaitons construire une structure donc nous voulons identifier 1 seul groupe car il faut que nos cubes soient tous "liés" entre eux.

Donc, tant que la place proposée pour le cube sépare notre structure nous en choisissons une autre.

Plusieurs paramètres sont définis pour controller notre algorithme :

- norm_value_contact, norm_value_free : are the values of norms to use to define proximity between the cube and others or between the cube and free spaces around
- str_norm_to_use : is the norm to use L1 or L2 to define this proximity
- print_progress : to print or not the graphs of cubes evolving in the while iteration. The while iteration tests whether there is too much groups of cubes or not
- connected_cubes_limit : to define the limit number of groups to admit

In [ ]:
# PARAMETERS : 
#    norm_value_contact, norm_value_free (values of norms to use to define proximity between cubes / free space)
#    str_norm_to_use (norm to use L1 or L2)
#    print_progress (to print or not the graphs of cubes evolving in the while iteration)
#    connected_cubes_limit (to define the limit number of groups to admit)
def P_3d(matrix):
    # we compute all the centered norms (for each cube in the matrix)
    norms = []
    for cube in range(1, int(np.max(matrix))+1):
        norm = centered_matrix_norm(matrix, cube)
        norms.append(norm)
    location_to_place_cube = (np.array([]),np.array([]),np.array([]))
    list_of_cubes = np.arange(np.max(matrix))+1
    while len(location_to_place_cube[0]) <= 0:
        cube = rng.choice(list_of_cubes, size=1)[0]
        list_of_cubes = np.delete( list_of_cubes, np.asarray(list_of_cubes == cube).nonzero()[0] )
        # TAKE TOO MUCH TIME :
        #   we identify location around cube :
        #   loc_around = np.full(matrix.shape, False)
        #   loc_around_true = np.asarray(norms[cube-1] <= 2).nonzero()
        #   for coord in range(len(loc_around_true[0])):
        #       loc_around[loc_around_true[0][coord]][loc_around_true[1][coord]][loc_around_true[2][coord]] = True
        # we identify location in contact with any other cube :
        loc_in_contact = np.full(matrix.shape, False)
        for block in range(1, int(np.max(matrix))+1):
            if block != cube:
                loc_in_contact_true = np.asarray(norms[block-1] <= norm_value_contact).nonzero()
                for coord in range(len(loc_in_contact_true[0])):
                    loc_in_contact[loc_in_contact_true[0][coord]][loc_in_contact_true[1][coord]][loc_in_contact_true[2][coord]] = True
        #location_to_place_cube = np.asarray( (matrix==0)&(loc_in_contact==True)&(loc_around==True) ).nonzero()
        location_to_place_cube = np.asarray( (matrix==0)&(loc_in_contact==True) ).nonzero()
    cube_coordinate = np.asarray( matrix==cube ).nonzero()
    connected_cubes = 10
    list_of_places = np.arange(len(location_to_place_cube[0]))
    limit = len(list_of_places)-3
    cptwhile = 0
    while connected_cubes > connected_cubes_limit:
        new_location_idx = rng.choice(list_of_places, size=1)[0]  
        list_of_places = np.delete( list_of_places, np.asarray(list_of_places == new_location_idx).nonzero()[0] ) # is the random place we choose in the location_to_place_cube arrays to be the new location of our cube
        cptwhile+=1
        new_x = location_to_place_cube[0][new_location_idx]
        new_y = location_to_place_cube[1][new_location_idx]
        new_z = location_to_place_cube[2][new_location_idx]
        # current place of the cube become 0 
        # new location become the identity integer of our cube
        temporary_matrix = np.copy(matrix)
        temporary_matrix[cube_coordinate[0][0]][cube_coordinate[1][0]][cube_coordinate[2][0]] = 0
        temporary_matrix[new_x][new_y][new_z] = cube
        Mcolor = colorate_new_matrix(temporary_matrix, dcolor)
        Mcolor[new_x][new_y][new_z] = 'black'
        graph = create_affinity_matrix(temporary_matrix, norm_value_contact, norm_value_free, str_norm_to_use)    
        if print_progress == True:
            if cptwhile > limit:    
                if cptwhile == limit+1 : # we just entered a new loop
                    print(f"\n\n\nTHE BLACK CUBE IS MOVING\n\n")
                print(f"We identify {connected_cubes} group(s) of cubes")
                ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
                ax.set_aspect('auto')
                ax.voxels(temporary_matrix, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
                plt.show()
        connected_cubes = connected_components(csgraph=graph, directed=False, return_labels=False)
    matrix = temporary_matrix
    new_cube_coords = np.array([new_x, new_y, new_z])
    old_cube_coord = np.array([cube_coordinate[0][0], cube_coordinate[1][0], cube_coordinate[2][0]])
    return matrix, new_cube_coords, old_cube_coord

Lançons l'algorithme :

Nous afficherons les étapes de la "construction" lorsque l'algorithme approche une impasse.

In [ ]:
norm_value_contact = 1
norm_value_free = 1
str_norm_to_use = "norm1"
print_progress = True
connected_cubes_limit = 1

if str_norm_to_use == "norm1":
    centered_matrix_norm = centered_matrix_norm1
elif str_norm_to_use == "norm2":
    centered_matrix_norm = centered_matrix_norm2
else :
    print("error norm, we will use norm1")
    centered_matrix_norm = centered_matrix_norm1
    
N0 = 100
distances, new_M = Metropolis(H, P_3d, T, M, N0)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
Mcolor = colorate_new_matrix(new_M, dcolor)
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/10))])
plt.title("Evolution de H au cours des itérations")
plt.show()

### Analyse step-by-step

THE BLACK CUBE IS MOVING

We identify 2 group(s) of cubes

<img src="simulation_9.png" width="500px">

We identify 2 group(s) of cubes

<img src="simulation_10.png" width="500px">

We identify 2 group(s) of cubes

<img src="simulation_11.png" width="500px">

Avez vous identifié le problème ?

Il arrive qu'un cube que l'on déplace sépare la structure, la faisant passer d'un seul et même groupe à 3 groupes distincts de cubes. Le cube que l'on déplace à ce moment ne peut jamais palier à ce problème sans revenir à sa position initiale (d'où on le déplace justement).

Nous avons donc plusieurs possibilité devant nous :

- Nous pouvons changer la norme définissant la proximité entre les cubes
- Nous pouvons ajouter une boucle dans notre processus pour changer de cube quand nous avons sélectionné un tel cube, nous faisant tomber dans une impasse (mais ca risque d'être très long)
- Nous pouvons tester avec plus ou moins de cubes
- Nous pouvons tester différentes matrices initiales (car le point initial impact l'evolution de notre algorithme)

Jusqu'à présent nous utilisions la norme 1 égale à 1 pour définir une proximité entre cubes.

Testons tout d'abord avec la norme 2 afin de considérer comme "en contact" des cubes qui se trouvent en diagonale.

In [ ]:
norm_value_contact = 1
norm_value_free = 1
str_norm_to_use = "norm2"
print_progress = True
connected_cubes_limit = 1

if str_norm_to_use == "norm1":
    centered_matrix_norm = centered_matrix_norm1
elif str_norm_to_use == "norm2":
    centered_matrix_norm = centered_matrix_norm2
else :
    print("error norm, we will use norm1")
    centered_matrix_norm = centered_matrix_norm1
       
N0 = 100
distances, new_M = Metropolis(H, P_3d, T, M, N0)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
Mcolor = colorate_new_matrix(new_M, dcolor)
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/10))])
plt.title("Evolution de H au cours des itérations")
plt.show()

### Analyse step-by-step

THE BLACK CUBE IS MOVING

We identify 2 group(s) of cubes

<img src="simulation_12.png" width="500px">

We identify 2 group(s) of cubes

<img src="simulation_13.png" width="500px">

We identify 2 group(s) of cubes

<img src="simulation_14.png" width="500px">

C'est difficile de garder une structure en 1 morceau avec cette solution...

Maintenant, toujours avec la norme 2, nous commençons avec une plus grosse quantité de cubes. En changeant ainsi la matrice initiale et la quantité de cubes nous espérons aboutir à un résultat plus convaincant.

In [ ]:
M = np.zeros((9,9,11))
c = 0 
for i in range(3,6):
    for j in range(3,6):
        M[i][j][0] = c+1
        M[i][j][1] = c+2
        M[i][j][2] = c+3
        M[i][j][3] = c+4
        M[i][j][4] = c+5
        M[i][j][5] = c+6
        M[i][j][6] = c+7
        M[i][j][7] = c+8
        c+=8
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
Mcolor, dcolor = create_colored_matrix(M, light_colors)
ax.voxels(M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

norm_value_contact = 1
norm_value_free = 1
str_norm_to_use = "norm2"
print_progress = True
connected_cubes_limit = 1

if str_norm_to_use == "norm1":
    centered_matrix_norm = centered_matrix_norm1
elif str_norm_to_use == "norm2":
    centered_matrix_norm = centered_matrix_norm2
else :
    print("error norm, we will use norm1")
    centered_matrix_norm = centered_matrix_norm1
    
N0 = 100
distances, new_M = Metropolis(H, P_3d, T, M, N0)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
Mcolor = colorate_new_matrix(new_M, dcolor)
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/10))])
plt.title("Evolution de H au cours des itérations")
plt.show()

<img src="simulation_15.png" width="500px">

<img src="simulation_16.png" width="500px">

<img src="H_evolution_4.png" width="500px">

Visiblement, l'algorithme ne rencontre pas d'erreur avec un plus grand nombre de cubes. Néanmoins, comparé aux situations précédentes nous pouvons voir sur ce graphique d'évolution de la fonction H que le minimum n'est pas atteint. L'algorithme met beaucoup de temps à s'éxécuter, il faudrait augmenter le nombre d'itérations. Le même problème persiste, avec la norme 2 il est impossible d'obtenir une structure viable.

Nous allons donc tenter d’utiliser la norme 2 uniquement pour identifier le nombre de groupes de cubes (des cubes en diagonale sont donc considérés comme étant en lien). Nous gardons la norme 1 pour le reste, c’est à dire la localisation des voisins des cubes, les surfaces en contact, l’identification des places libres autour... La norme 2 interviendra uniquement dans le calcul de la matrice de liens pour définir les affinités entre cubes. (Pour cela nous définissons à nouveau la fonction "create_affinity_matrix").

In [ ]:
def P_3d(matrix):
    # we compute all the centered norms (for each cube in the matrix)
    norms = []
    for cube in range(1, int(np.max(matrix))+1):
        norm = centered_matrix_norm(matrix, cube)
        norms.append(norm)
    location_to_place_cube = (np.array([]),np.array([]),np.array([]))
    list_of_cubes = np.arange(np.max(matrix))+1
    while len(location_to_place_cube[0]) <= 0:
        cube = rng.choice(list_of_cubes, size=1)[0]
        list_of_cubes = np.delete( list_of_cubes, np.asarray(list_of_cubes == cube).nonzero()[0] )
        # TAKE TOO MUCH TIME :
        #    we identify location around cube :
        #    loc_around = np.full(matrix.shape, False)
        #    loc_around_true = np.asarray(norms[cube-1] <= 2).nonzero()
        #    for coord in range(len(loc_around_true[0])):
        #        loc_around[loc_around_true[0][coord]][loc_around_true[1][coord]][loc_around_true[2][coord]] = True
        # we identify location in contact with any other cube :
        loc_in_contact = np.full(matrix.shape, False)
        for block in range(1, int(np.max(matrix))+1):
            if block != cube:
                loc_in_contact_true = np.asarray(norms[block-1] <= norm_value_contact).nonzero()
                for coord in range(len(loc_in_contact_true[0])):
                    loc_in_contact[loc_in_contact_true[0][coord]][loc_in_contact_true[1][coord]][loc_in_contact_true[2][coord]] = True
        #location_to_place_cube = np.asarray( (matrix==0)&(loc_in_contact==True)&(loc_around==True) ).nonzero()
        location_to_place_cube = np.asarray( (matrix==0)&(loc_in_contact==True) ).nonzero()
    cube_coordinate = np.asarray( matrix==cube ).nonzero()
    connected_cubes = 10
    list_of_places = np.arange(len(location_to_place_cube[0]))
    limit = len(list_of_places)-3
    cptwhile = 0
    while connected_cubes > connected_cubes_limit:
        new_location_idx = rng.choice(list_of_places, size=1)[0] 
        list_of_places = np.delete( list_of_places, np.asarray(list_of_places == new_location_idx).nonzero()[0] ) # is the random place we choose in the location_to_place_cube arrays to be the new location of our cube
        cptwhile+=1
        new_x = location_to_place_cube[0][new_location_idx]
        new_y = location_to_place_cube[1][new_location_idx]
        new_z = location_to_place_cube[2][new_location_idx]
        # current place of the cube become 0 
        # new location become the identity integer of our cube
        temporary_matrix = np.copy(matrix)
        temporary_matrix[cube_coordinate[0][0]][cube_coordinate[1][0]][cube_coordinate[2][0]] = 0
        temporary_matrix[new_x][new_y][new_z] = cube
        Mcolor = colorate_new_matrix(temporary_matrix, dcolor)
        Mcolor[new_x][new_y][new_z] = 'black'
        graph = create_affinity_matrix(temporary_matrix, norm_value_contact, norm_value_free, "norm2")    
        if print_progress == True:
            if cptwhile > limit:    
                if cptwhile == limit+1 : # we just entered a new loop
                    print(f"\n\n\nTHE BLACK CUBE IS MOVING\n\n")
                print(f"We identify {connected_cubes} group(s) of cubes")
                ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
                ax.set_aspect('auto')
                ax.voxels(temporary_matrix, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
                plt.show()
        connected_cubes = connected_components(csgraph=graph, directed=False, return_labels=False)
    matrix = temporary_matrix
    new_cube_coords = np.array([new_x, new_y, new_z])
    old_cube_coord = np.array([cube_coordinate[0][0], cube_coordinate[1][0], cube_coordinate[2][0]])
    return matrix, new_cube_coords, old_cube_coord

In [ ]:
norm_value_contact = 1
norm_value_free = 1
str_norm_to_use = "norm1"
print_progress = True
connected_cubes_limit = 1
  
N0 = 500
distances, new_M = Metropolis(H, P_3d, T, M, N0)
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
ax.set_aspect('auto')
Mcolor = colorate_new_matrix(new_M, dcolor)
ax.voxels(new_M, facecolors = Mcolor, edgecolor = "white", alpha = 0.8)
plt.show()

plt.figure(figsize=(15,7))
plt.plot([distances[i] for i in range(0, N0, int(N0/10))])
plt.title("Evolution de H au cours des itérations")
plt.show()

THE BLACK CUBE IS MOVING

We identify 3 group(s) of cubes

<img src="simulation_17.png" width="500px">

Encore une fois, le code est très long et rencontre une erreur avant la fin...

Pour conclure, nous avons obtenu la meilleure combinaison avec la norme 1 et un "grand" nombre de cubes. Voici une matrice initiale accompagné de 2 résultats obtenus lors de différentes executions du code.

<img src="simulation_18.png" width="500px">

<img src="simulation_19.png" width="500px">

Ces résultats finaux correspondent bien à nos objectifs mêmes s'ils ne représentent pas le résultat optimal atendu minimisant la fonction objectif.

Finalement, pour répondre à notre problématique initial, il semble important d'optimiser son code car la procédure du recuit simulé peut être très longue. De plus, comme nous l'avons vu, la définition de la fonction objectif à minimiser est une étape importante et complexe de ce projet. Globalement sur les 2 résultats finaux présentés ci-dessus, une structure arborescente apparaît. Un problème éventuel serait le fait que des cubes s'accrochent côte à côte sans qu'aucun cube ne soit présent en dessous ; ce qui n'est pas possible, il faudrait donc prendre en compte cet aspect là dans la construction si nous voulions continuer ce projet.

Néanmoins, les résultats obtenus sont en accord avec les contraintes définies. L'objectif de ce projet est ainsi en parti atteint et laisse place à de nouvelles perspectives pour prolonger et approfondir cette recherche.